In [1]:
import json

from utilities.visualize import create_error_timeline_gif
from utilities.ArUco_detector import convert_predictions, compare_sequences

In [2]:
# Ground truth data has 12 joints, their coordinates are given in meters (m)
# Ground truth data might be missing some of the joint positions in some frames (of the sequence)
ground_truth_json_path = "ground_truth/seq_003_j12_t20260212_164457.json"

# Ground truth data has 15 joints, their coordinates are given in milimeters (mm)
# For all of the frames in the sequence if a prediction is made every joint is predicted and exists in the JSON
# However in some frames the prediction was not cpnfident enough so there are no prediction at all -> "181": {}
prediction_json_path = "predictions/seq_003_j15_t20260211_232604.json"


# The mapping shows the mapping from predicted frames to ground truth data, only 10 of the mappings can be made
# It maps like (joint_idx_predicted, joint_idx_truth) ,n the list below
joint_mapping_pairs = [(11, 0), (10, 1), (9, 2), (5 , 3), (4, 4), (3, 5), (14, 6), (13, 7), (8, 8), (7, 9)]

with open(ground_truth_json_path, 'r') as f:
    ground_truth_data = json.load(f)

with open(prediction_json_path, 'r') as f:
    prediction_data = json.load(f)

In [3]:
processed_preds = convert_predictions(prediction_data, joint_mapping_pairs)
differences = compare_sequences(ground_truth_data, processed_preds)

In [4]:
# Output results
print("Differences (Euclidean Distance in Meters) per Frame per Joint:")
for frame_id, diff_data in differences.items():
    print(f"\nFrame {frame_id}:")
    total_error = 0
    count = 0
    for joint_id, dist in diff_data.items():
        print(f"  Joint {joint_id}: {dist:.4f} m")
        total_error += dist
        count += 1
    if count > 0:
        print(f"  [Average Error for Frame {frame_id}]: {total_error/count:.4f} m")

# Print processed prediction sample to verify transform
print("\n--- Verification Sample ---")
print(f"GT Frame 0 Joint 0: {ground_truth_data['0']['0']}")
print(f"Pred Frame 0 Joint 0 (mapped from 11): {processed_preds['0']['0']}")

Differences (Euclidean Distance in Meters) per Frame per Joint:

Frame 0:
  Joint 0: 0.3247 m
  Joint 1: 0.3293 m
  Joint 2: 0.3456 m
  Joint 3: 0.2655 m
  Joint 4: 0.3207 m
  Joint 5: 0.2795 m
  Joint 6: 0.3765 m
  Joint 7: 0.3125 m
  Joint 8: 0.2688 m
  Joint 9: 0.3081 m
  [Average Error for Frame 0]: 0.3131 m

Frame 1:
  Joint 0: 0.3233 m
  Joint 1: 0.3315 m
  Joint 2: 0.3514 m
  Joint 3: 0.2672 m
  Joint 4: 0.3186 m
  Joint 5: 0.2834 m
  Joint 6: 0.3743 m
  Joint 7: 0.3125 m
  Joint 8: 0.2646 m
  Joint 9: 0.3094 m
  [Average Error for Frame 1]: 0.3136 m

Frame 2:
  Joint 0: 0.3253 m
  Joint 1: 0.3392 m
  Joint 2: 0.3569 m
  Joint 3: 0.2717 m
  Joint 4: 0.3202 m
  Joint 5: 0.2819 m
  Joint 6: 0.3711 m
  Joint 7: 0.3165 m
  Joint 8: 0.2651 m
  Joint 9: 0.3139 m
  [Average Error for Frame 2]: 0.3162 m

Frame 3:
  Joint 0: 0.3231 m
  Joint 1: 0.3397 m
  Joint 2: 0.3594 m
  Joint 3: 0.2727 m
  Joint 4: 0.3186 m
  Joint 5: 0.2761 m
  Joint 6: 0.3734 m
  Joint 7: 0.3293 m
  Joint 8: 0.263

In [5]:
# differences['181'] = {}
# differences['10'] = {'0': 0.3258322720303928, '1': 0.3666073536346595, '2': 0.36203606036283115, '3': 0.34614295465878947, '4': 0.40464295926617533, '5': 0.32974768577531655, '6': 0.47641542240512097, '7': 0.3449455639403206, '8': 0.2885587394577386, '9': 0.3458007312672581}
# create_error_timeline_gif(differences, "seq002_error_graph.gif", duration_ms=70)

{}
{'0': 0.3258322720303928, '1': 0.3666073536346595, '2': 0.36203606036283115, '3': 0.34614295465878947, '4': 0.40464295926617533, '5': 0.32974768577531655, '6': 0.47641542240512097, '7': 0.3449455639403206, '8': 0.2885587394577386, '9': 0.3458007312672581}
Generating GIF with 207 frames...
Saved to seq002_error_graph.gif
